# Numerical exercise
Kristine Schüller and Sigrid Aunsmo

In [ ]:
h_bar=1
m=1
k_0=20
L=20
t_0=0

## problem 1

## problem 2

In [ ]:
x_s=5
L=2*10
x_end=15

V_g=(h_bar*k_0)/m

## problem 3

In [ ]:
l=L/50
V0=E/2

def V(x):
    if (L/2-l/2<x)or(L/2+l/2>x):
        return E/2
    else: 
        return 0

## problem 4

In [ ]:


"Making a list of 50 different values of V0"
V0_list=np.arange(0,(3*E)/2,50)


## problem 5

In [ ]:
"Making a list of 50 different values of L"
L_list=np.arange(0,L/20,50)
